# 简单的 RAG 应用

本节我们完成一个简单的 RAG 应用。我们将一个文档向量化后，存入向量数据库中，然后用 `deepseek-r1:1.5b` 模型，整合 RAG 取回的内容后输出回答。

最终的 langflow 工作流如下：

![](./img/simple_rag_app.jpg)

## 1. 环境准备

搭这个应用，最难的反而是搭环境这一步 (`ヮ´ )

### 1.1 安装 Ollama

1）下载 Ollama

下载 Ollama：[ollama.com/download](https://ollama.com/download)

它提供了 macOS, Linux, Windows 三种系统的下载方式。直接按你的系统安装就行。Windows 电脑建议直接安装，不用多此一举安装到 wsl。

Linux 用户用以下命令安装：

```bash
curl -fsSL https://ollama.com/install.sh | sh
```

查看版本号：

```bash
ollama --version
```

2）下载模型

下载词向量模型 bge-m3 和推理模型 deepseek-r1:1.5b

```bash
# 下载 bge-m3
ollama pull bge-m3

# 下载 deepseek-r1:1.5b
ollama pull deepseek-r1:1.5b
```

列出所有模型：

```bash
ollama list
```

3）启动 deepseek-r1

```bash
ollama run deepseek-r1:1.5b
```

启动后，你可以在命令行与 deepseek-r1 交互。

PS: 更多命令我放在项目目录下的 [./deploy/install_ollama.sh](./deploy/install_ollama.sh)

### 1.2 安装 langflow

1）安装

如果你是 Windows 电脑，建议把 langflow 安装到 wsl 的 Ubuntu 环境。因为 Windows 的 Python 环境管理实在是抽象。

```bash
# 创建 conda 环境
conda create -n langflow python=3.11 -y

# 激活 conda 环境
conda activate langflow

# 下载 langflow 通过阿里云镜像站
pip install langflow -U -i https://mirrors.aliyun.com/pypi/simple/

# 运行 langflow
langflow run
```

安装 langflow 的过程会有点久。详情参考：[docs.langflow.org](https://docs.langflow.org/)

2）快速启动

我创建了一个 langflow 的快速启动脚本，用来快速切换到 langflow 的 Conda 环境并启动 langflow：

```bash
#!/bin/bash
source $(conda info --base)/etc/profile.d/conda.sh
conda activate langflow
langflow run
```

程序文件见 [./langflow_start.sh](./langflow_start.sh)

快速启动 langflow 在命令行运行：

```bash
bash langflow_start.sh
# 或 ./langflow_start.sh
```

### 1.3 安装 chroma

```bash
conda activate langflow
pip install chromadb -i https://mirrors.aliyun.com/pypi/simple/
```

## 2. langflow 搭建工作流

启动 langflow 需要一点的时间：

```bash
# 如果是 Windows 电脑，确保先打开 wsl
bash langflow_start.sh
```

浏览器打开 langflow 的默认端口 http://127.0.0.1:7860/ 进入 UI 界面。

![](./img/langflow_start.jpg)

1）创建一个新的 Flow

点击 `+ New Flow` 选择 `Q&A` 分页，然后点击 `Vector Store RAG`

![](./img/new_rag_flow.jpg)

2）初始界面

现在它提供给你的已经是一个完整的 RAG 工作流。可以看到这个工作流分上、下两块：

- 上面是把用户问题向量化，然后到向量库取回相关文本，再使用提示词模板结合用户问题与相关文本，送入大模型的过程
- 下面是把文本切分成文本片段，转成向量，然后存入向量数据库的过程

![](./img/simple_rag_app_init.jpg)

3）本地改造计划

下面来改造它。它有三个地方要改：Embedding 方法要改成本地的 bge-m3；Astra DB 也要换成本地的 chroma；OpenAI 的模型也要换成本地的 deepseek-r1。

4）启动 Ollama 服务

因为 bge-m3 和 deepseek-r1 都是由 Ollama 支持的。因此要先启动 Ollama 服务.

我是 Windows 电脑，Ollama 在本机，但是 langflow 在 wsl，如果在 `127.0.0.1` 启动 Ollama 服务，wsl 是访问不到的，必须在 `0.0.0.0` 启动才行。因此启动前需要配置环境变量：

```bash
# 对于 Windows PowerShell
$env:OLLAMA_HOST = "0.0.0.0:11434"
ollama serve
```

对于 Linux 应该使用 `export OLLAMA_HOST=0.0.0.0:11434` 配置环境变量。

> PS: 此时在浏览器访问 [http://127.0.0.1:11434/](http://127.0.0.1:11434/)，应该可以看到显示 `Ollama is running` 这几个字。

5）添加 Ollama Embeddings 组件

我们在侧边栏找到 `Ollama Embeddings` 这个组件，点击加号添加组件。

![](./img/ollama_embed.jpg)

然后我们在 `Ollama Base URL` 这里，在下拉栏中选择 `Add New Variable`，选择 Generic 页面，填 Name 和 value

- **Name**: `ollama_url`
- **Value**: `http://[YOUR_IP_HERE]:11434`

> **[YOUR_IP_HERE] 的填法：**
> 
> 1.如果你不是 Windows 电脑，也没有在 wsl 上打开 langflow，[YOUR_IP_HERE] 填 `127.0.0.1` 就好。
> 
> 2.如果你是 Windows 电脑，并且在 wsl 打开 langflow，你需要获取本机 ip：
> 
> ```bash
> # Windows 电脑
> ipconfig
> ```
>
> 比如我的是 `192.168.2.192`，就在上面的 [YOUR_IP_HERE] 填上 `192.168.2.192`。

填好后点击 Save Variable.

![](./img/ollama_variable.jpg)

然后在 Ollama Model 这里选 `bge-m3:latest`

再复制一个 Ollama Embeddings 组件，把两个 OpenAI Embeddings 组件都替换掉。

6）添加 Chroma DB 组件

- **Collection Name**: `rag_simple_app`
- **Persist Directory**: 新建一个 Generic 变量，Name 为 `chroma_path`，Value 为 `./chroma_db/`

复制 Chroma DB 组件，替换掉两个 Astra DB。

第一个 Chroma DB：

![](./img/add_chroma_db_1.jpg)

第二个 Chroma DB：

![](./img/add_chroma_db_2.jpg)

7）添加 Ollama 组件

添加一个 Ollama 组件，替换掉 OpenAI 组件。

Model Name 设为 deepseek-r1:1.5b，Temperature 参数设为 0.6

![](./img/add_ollama.jpg)

8）传入文档

传入文档 [《人工智能伦理指南》](./docs/test_rag.md)，这是我让 qwen2.5-max 编的，用来测试 RAG 功能的文档。

然后点击下方的 Chroma DB 的运行按钮 `▶`，这样就完成了文档的向量化，并将文本向量存入了 Chroma DB。

再点开 Playground，我们在这里向 ChatBot 提问。

![](./img/simple_rag_playground.jpg)

可以看到，它准确地回答了我们编造的内容。

由此可见，大模型真的能够读到我们上传的文档。